In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import inflect

In [ ]:
# English inflection helper (used to detect singular/plural of input terms)
p = inflect.engine()

In [ ]:
SPANISH_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Todos los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sin excepción, todos los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cada {a_sg} es {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Todo lo que es {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Cada uno de los {a_pl} pertenece a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si algo es {a_sg_art}, entonces es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La clase de los {a_pl} está incluida en la de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cualquier {a_sg} califica como {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'No hay ningún {a_sg} que no sea {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Ser {a_sg_art} implica ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La totalidad de los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es un hecho que cada {a_sg} es {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Todo miembro de {a_pl} es también miembro de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cualquiera de los {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'No existe un {a_sg} que no sea a la vez {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'}
        ],
        'some': [
            {'tpl': 'Algunos {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existe al menos un {a_sg} que es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Hay {a_pl} que son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Al menos algunos {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Unos pocos {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ciertos {a_pl} resultan ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Al menos una instancia de {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Es posible encontrar {a_pl} que sean {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Una parte de los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existen casos donde {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'No todos, pero sí algunos {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Entre los {a_pl}, se encuentran {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Hay al menos un ejemplo de {a_sg_art} que es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Varios {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Se da el caso de que algunos {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Ningún {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'No existe {a_sg} que sea {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Nada que sea {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'No hay {a_pl} que sean {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Jamás los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ninguno de los {a_pl} pertenece a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es falso que algún {a_sg} sea {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'La clase de los {a_pl} es ajena a la de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Bajo ninguna circunstancia un {a_sg} es {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Absolutamente ningún {a_sg} es {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Queda excluido que los {a_pl} sean {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si es {a_sg_art}, no puede ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'No es posible hallar un {a_sg} que sea {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Ningún miembro de {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'La intersección entre {a_pl} y {b_pl} es vacía.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Algunos {a_pl} no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'No todo {a_sg} es {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Algunos {a_pl} no pertenecen a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'No todos los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Hay {a_pl} que no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ciertos {a_pl} carecen de la cualidad de ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existe al menos un {a_sg} que no es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'No todos los miembros de {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Hay al menos una instancia de {a_sg_art} que no es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'No es el caso que cada {a_sg} sea {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Parte de los {a_pl} no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Algunos de los {a_pl} están fuera de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Podemos encontrar {a_pl} que no resultan ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'No toda cosa que sea {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Al menos algunos {a_pl} son ajenos a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Por lo tanto, {conclusion}.',
        'En consecuencia, {conclusion}.',
        'Así que {conclusion}.',
        'Por consiguiente, {conclusion}.',
        'De ello se sigue que {conclusion}.',
        'Se deduce que {conclusion}.',
        'En conclusión, {conclusion}.',
        'Queda claro que {conclusion}.',
        'De esto resulta que {conclusion}.',
        'Por ende, {conclusion}.'
    ],
}

In [ ]:
SPANISH_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Cada {d_sg} que es {a_sg_art} es también {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Si {d_sg_art} es {a_sg_art}, entonces es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Todos los {d_pl} que son {a_pl} también son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Sin excepción, los {d_pl} que son {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Cada {d_sg} perteneciente a {a_sg_art} resulta ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Cualquier {d_sg} que califique como {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Todo {d_sg} con la propiedad de ser {a_sg} es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Si un objeto es {d_sg_art} y {a_sg_art}, necesariamente es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'La totalidad de los {d_pl} que son {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Dentro de {d_pl}, todos los {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Para cualquier {d_sg}, si es {a_sg_art} entonces es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Todo lo que sea a la vez {d_sg_art} y {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Cualquiera de los {d_pl} que sea {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Cada miembro de {d_pl} que es {a_pl} pertenece a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'No hay ningún {d_sg} que siendo {a_sg_art} no sea {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'}
        ],
        'some': [
            {'tpl': 'Existe al menos un {d_sg} que es tanto {a_sg_art} como {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Algunos {d_pl} son {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Hay {d_pl} que son tanto {a_pl} como {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Al menos uno de los {d_pl} es {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Un {d_sg} existe que es {a_sg} y {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Ciertos {d_pl} resultan ser {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es posible hallar un {d_sg} que sea {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Se da el caso de que algún {d_sg} es {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Una parte de los {d_pl} son a la vez {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Hay al menos un ejemplo de {d_sg} que es {a_sg} y {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Podemos encontrar {d_pl} que son {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Algún {d_sg} es simultáneamente {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Dentro de {d_pl}, existen {a_pl} que son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Hay {d_pl} que cumplen con ser {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Varios {d_pl} son {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Ninguno de los {d_pl} que son {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'No hay {d_pl} que sean {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Jamás los {d_pl} que son {a_pl} son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'No existe {d_sg} que sea {a_sg} y {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Ningún {d_sg} que sea {a_sg} es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Es imposible que un {d_sg} sea {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ningún {d_sg} con la cualidad {a_sg} es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'No existe ningún {d_sg} que sea simultáneamente {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Dentro de {d_pl}, ningún {a_pl} es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Queda excluido que los {d_pl} sean {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es falso que algún {d_sg} sea a la vez {a_sg_art} y {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ninguno de los miembros de {d_pl} es {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Si es {d_sg_art} y {a_sg_art}, entonces no es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'No hay forma de que un {d_sg} sea {a_sg} y {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'La categoría {d_pl} no admite la combinación {a_pl} y {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'No todo {d_sg} que es {a_sg_art} es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Hay {d_pl} que son {a_pl} pero no {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Algunos {d_pl} que son {a_pl} no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Al menos uno de los {d_pl} que es {a_pl} no es {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Un {d_sg} que sea {a_sg} puede no ser {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Ciertos {d_pl} que son {a_pl} carecen de ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Existe al menos un {d_sg} que siendo {a_sg_art} no es {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'No es el caso que cada {d_sg} que es {a_sg_art} sea {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Hay miembros de {d_pl} que son {a_pl} pero están fuera de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Podemos hallar un {d_sg} que es {a_sg} pero no es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Dentro de {d_pl}, algunos {a_pl} no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'No todo lo que es {d_sg_art} y {a_sg_art} resulta ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Algún {d_sg} que es {a_sg} no es {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Es posible que un {d_sg} sea {a_sg} y sin embargo no sea {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Varios {d_pl} en {a_pl} no son {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    """Return (singular, plural) using English inflect heuristics.
    Falls back to term if detection fails.
    """
    term = term.strip()
    try:
        sg_attempt = p.singular_noun(term)
    except Exception:
        sg_attempt = False
    if sg_attempt is not False and sg_attempt is not None:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        try:
            pl = p.plural_noun(term) or term
        except Exception:
            pl = term
    return sg, pl

In [ ]:
def get_article_spanish(term):
    """Simple Spanish indefinite article heuristic: 'una' if word ends with 'a', else 'un'.
    This is a heuristic and does not attempt full gender agreement or pluralization.
    """
    term = term.strip()
    if term.endswith('a'):
        return 'una ' + term
    return 'un ' + term

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    match = re.match(r'All (.*) that are (.*) are (.*).', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*).', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*).', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*).', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    match = re.match(r'All (.*) are (.*).', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*).', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*).', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*).', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse basic syllogism type from sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article_spanish(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article_spanish(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article_spanish(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if isinstance(val, str) and s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f'Syllogism must contain at least one premise and one conclusion: {syllogism}')
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f' Skipping item {item.get(id, 'N/A')} due to parse error: {e}')
            continue
    return complex_dataset

In [ ]:
# Example English input data
sample_data = [
    {
        'id': 'sp-1',
        'syllogism': 'All dogs are animals. All animals are living things. Some dogs are living things.',
        'validity': True,
        'plausibility': True
    },
    {
        'id': 'sp-2',
        'syllogism': 'No fish are mammals. Some mammals are whales. Some whales are not fish.',
        'validity': True,
        'plausibility': True
    }
]
# Run conversion using Spanish templates
complex_syllogism_data = process_dataset(sample_data, SPANISH_COMPLEX_PATTERNS, SPANISH_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))